<a href="https://colab.research.google.com/github/SoumikSarkar830/Laser_PINN/blob/main/LaserPINN_Zigzag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
class PINN(nn.Module):
    def __init__(self):
      super(PINN, self).__init__()
      self.net = nn.Sequential(
          nn.Linear(3, 100),  # x,y,t as inputs
          nn.Tanh(),
          nn.Linear(100, 100),
          nn.Tanh(),
          nn.Linear(100, 100),
          nn.Tanh(),
          nn.Linear(100, 100),
          nn.Tanh(),
          nn.Linear(100, 1)  # Temparaure T as output
      )

    def forward(self, x):
      return self.net(x)


In [4]:
model = PINN()
model.to(device)

PINN(
  (net): Sequential(
    (0): Linear(in_features=3, out_features=100, bias=True)
    (1): Tanh()
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): Tanh()
    (4): Linear(in_features=100, out_features=100, bias=True)
    (5): Tanh()
    (6): Linear(in_features=100, out_features=100, bias=True)
    (7): Tanh()
    (8): Linear(in_features=100, out_features=1, bias=True)
  )
)

In [5]:
stefanBoltz = 5.670373e-8
hCoeff = 1
emiss = .5
ta = 300

In [6]:
def heat_loss(model, x, y, t, alpha = 0.01):

    # PDE Loss

    inputs = (torch.cat((x, y, t), dim=1))
    T = model(inputs)

    dT_dx = torch.autograd.grad(T, x, torch.ones_like(T), create_graph=True)[0]
    dT_dy = torch.autograd.grad(T, y, torch.ones_like(T), create_graph=True)[0]
    dT_dt = torch.autograd.grad(T, t, torch.ones_like(T), create_graph=True)[0]

    dT_dx2 = torch.autograd.grad(dT_dx, x, torch.ones_like(dT_dx), create_graph=True)[0]
    dT_dy2 = torch.autograd.grad(dT_dy, y, torch.ones_like(dT_dy), create_graph=True)[0]

    laser_heat = 2 * (emiss * stefanBoltz * (T**4 - ta**4) + hCoeff * (T - ta))

    residual = dT_dt - alpha * (dT_dx2 + dT_dy2) - laser_heat
    pde_loss = torch.mean(residual**2)


    # Dirichlet Boundary Conditions

    fixed_T = 100.0
    bc_x = (x == 0) | (x == 1)
    bc_y = (y == 0) | (y == 1)

    bc_loss = torch.mean((model(torch.cat([x[bc_x].reshape(-1, 1), y[bc_x].reshape(-1, 1), t[bc_x].reshape(-1, 1)], dim=1)) - fixed_T)**2)

    # Neumann Boundary Conditions

    neumann_loss_x = torch.mean(dT_dx[bc_x]**2)
    neumann_loss_y = torch.mean(dT_dy[bc_y]**2)
    neumann_loss = neumann_loss_x + neumann_loss_y

    return pde_loss + bc_loss + neumann_loss








In [7]:
grid_size = 100
time_steps = 100

x_data, y_data, t_data = [],[],[]
laser_path = []

for i in range(grid_size // 5):
    y = i / (grid_size // 5)
    # Left to right
    for x in np.linspace(0, 1, grid_size):
        laser_path.append((x, y))
    # Right to left for the next hatch if there is one
    if i + 1 < grid_size // 5:
        i += 1
        y = i / (grid_size // 5)
        for x in np.linspace(1, 0, grid_size):
            laser_path.append((x, y))

for t in range(time_steps):
    for pos in laser_path:
        x_data.append(pos[0])
        y_data.append(pos[1])
        t_data.append(t / time_steps)  # Normalize time

x_data = torch.tensor(x_data, dtype=torch.float32).view(-1, 1).requires_grad_(True).to(device)
y_data = torch.tensor(y_data, dtype=torch.float32).view(-1, 1).requires_grad_(True).to(device)
t_data = torch.tensor(t_data, dtype=torch.float32).view(-1, 1).requires_grad_(True).to(device)

In [8]:
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
epochs = 10000

for epoch in range(epochs):
    optimizer.zero_grad()
    loss = heat_loss(model, x_data, y_data, t_data)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

# Testing the trained model

x_test = torch.linspace(0, 1, 100).view(-1, 1).to(device)
y_test = torch.linspace(0, 1, 100).view(-1, 1).to(device)
t_test = torch.zeros_like(x_test).to(device)

with torch.no_grad():
    T_train_pred = model(torch.cat((x_data, y_data, t_data), dim=1))



plt.figure(figsize=(8, 8))
plt.scatter(x_data.detach().cpu().numpy(), y_data.detach().cpu().numpy(), c=T_train_pred.detach().cpu().numpy(), cmap='viridis')
plt.colorbar(label='Temperature')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Temperature Distribution')
plt.show()

Epoch 0, Loss: 1132026.0
Epoch 100, Loss: 718775.0625
Epoch 200, Loss: 318554.1875
Epoch 300, Loss: 44113.65234375
Epoch 400, Loss: 39389.3515625
Epoch 500, Loss: 39389.21875
Epoch 600, Loss: 39389.21875
Epoch 700, Loss: 39389.21875
Epoch 800, Loss: 39389.21875
Epoch 900, Loss: 39389.21875
Epoch 1000, Loss: 39389.21875
Epoch 1100, Loss: 39389.21875
Epoch 1200, Loss: 39389.21875
Epoch 1300, Loss: 39389.21875
Epoch 1400, Loss: 39389.21875
Epoch 1500, Loss: 39389.21875
Epoch 1600, Loss: 39389.21875
Epoch 1700, Loss: 39389.21875
Epoch 1800, Loss: 39389.21875
Epoch 1900, Loss: 39389.21875
Epoch 2000, Loss: 39389.21875
Epoch 2100, Loss: 39389.21875
Epoch 2200, Loss: 39389.21875
Epoch 2300, Loss: 39389.21875
Epoch 2400, Loss: 39389.21875
Epoch 2500, Loss: 39389.21875
Epoch 2600, Loss: 39389.21875
Epoch 2700, Loss: 39389.21875
Epoch 2800, Loss: 39389.21875
